# Multiple Human Phenotype Ontology visualizations
In this tutorial, we will visualize the Human Phenotype Ontology graph using all of the available embedding methods that can embed it.

This will exclude methods that require edge weights.

## Installing 🍇
To install GRAPE, run:

In [ ]:
! pip install grape -Uq

🍇 provides wrapping for the Karate Club and PyKEEN libraries, but if you want to use them you'll need to install them separately.

In [ ]:
! pip install pykeen karateclub -Uq

## Retrieving the graph

In [ ]:
from grape.datasets.kgobo import HP
from grape import GraphVisualizer
graph = HP()

To get the graph report, just run:

In [ ]:
graph

Since the graph contains disconnected nodes, including singleton nodes with selfloops, we need to drop them:

In [ ]:
graph = graph.remove_disconnected_nodes()

## Split the graph into train and test holdouts

Since we intend to visualize edge types and edge predictions, we execute a connected holdout splitting the edges 70/30. In a connected holdout, the training set will maintain the same number of connected components as the original graph.

This is necessary as topological edge prediction methods are not able to predict edges between disconnected components, and it would be a futile exercise.

In [ ]:
%%time
train, test = graph.connected_holdout(train_size=0.7)

In [ ]:
train.enable()

## Retrieving the available embedding methods

In [ ]:
from grape import get_available_models_for_node_embedding

all_embedding_methods = get_available_models_for_node_embedding()
embedding_methods = all_embedding_methods[~all_embedding_methods.requires_edge_weights.astype(bool)]

# We remove some models that either require some additional
# information or are simply currently work in progress.
# We also remove some models because they are too slow,
# or die because of memory requirements.
embedding_methods = embedding_methods[
    ~embedding_methods.model_name.str.contains("SPINE") &
    ~embedding_methods.model_name.str.contains("WINE") &
    ~(
        (embedding_methods.model_name == "NodeSketch") &
        (embedding_methods.library_name == "Karate Club") |
        (embedding_methods.model_name == "Role2Vec") &
        (embedding_methods.library_name == "Karate Club") |
        (embedding_methods.model_name == "Node2Vec SkipGram") &
        (embedding_methods.library_name == "Karate Club") |
        (embedding_methods.model_name == "Walklets SkipGram") &
        (embedding_methods.library_name == "Karate Club") |
        (embedding_methods.model_name == "CP") &
        (embedding_methods.library_name == "PyKEEN") 
    )
]

## Running the visualizations

In [ ]:
import silence_tensorflow.auto
from grape import GraphVisualizer
from grape.utils import AbstractEmbeddingModel
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from math import ceil

ncols = 8
nrows = ceil(embedding_methods.shape[0] / ncols)

plots = [
    plt.subplots(nrows=nrows, ncols=ncols, figsize=(6*ncols, 6*nrows))
    for i in range(9)
]

for index, (_, row) in tqdm(
    enumerate(embedding_methods.iterrows()),
    total=embedding_methods.shape[0],
    desc="Computing embedding"
):
    model = AbstractEmbeddingModel.get_model_from_library(
        model_name=row.model_name,
        library_name=row.library_name
    )(enable_cache=True)
    embedding = model.fit_transform(
        train,
        return_dataframe=False
    )
    # We visualize the edges of the test graph,
    # but we use the complete graph as topological
    # support to compute the visualized properties,
    # such as the node degree, adamic adar and so on.
    vis = GraphVisualizer(
        graph=test,
        support=graph,
        node_embedding_method_name=f"{row.model_name} {row.library_name}",
        automatically_display_on_notebooks=False
    )
    vis.fit_negative_and_positive_edges(embedding)
    vis.fit_nodes(embedding)
    for callback, (fig, axes) in zip(
        (
            vis.plot_node_degrees,
            vis.plot_node_ontologies,
            vis.plot_edge_types,
            vis.plot_positive_and_negative_edges,
            vis.plot_positive_and_negative_edges_adamic_adar,
            vis.plot_positive_and_negative_edges_jaccard_coefficient,
            vis.plot_positive_and_negative_edges_preferential_attachment,
            vis.plot_positive_and_negative_edges_resource_allocation_index
        ),
        plots
    ):
        callback(
            figure=fig,
            axes=axes[index // ncols, index % ncols]
        )

for (fig, axes) in plots:
    fig.tight_layout()
    for axis in axes.flatten()[index:]:
        for spine in axis.spines.values():
            spine.set_visible(False)
        axis.axis("off")

plt.show()